In [1]:
import pandas as pd
import numpy as np
import os
import gzip
import io
from scripts.check_overlaps import check_overlaps, check_overlaps_and_annotate

Rheinbay 2020 Supplementary Table 5<br>
We filter on the basis of 'Final filter judgement' = TRUE<br>
Then we take the unique element IDs<br>

In [2]:
df = pd.read_excel('data/Rheinbay 2020/41586_2020_1965_MOESM4_ESM.xlsx', engine='openpyxl', sheet_name='Table 5 Non-coding point mut dr')
# df1 = pd.read_excel('data/Rheinbay 2020/41586_2020_1965_MOESM4_ESM.xlsx', engine='openpyxl', sheet_name='Table 4 Protein-coding point mu')
df['file_name'] = df['ID'].apply(lambda x: x.split('::')[0]+'.bed')
df['file_name'] = df['file_name'].apply(lambda x: x.replace('mirna_', 'mirna.') if 'mirna_pre' in x or 'mirna_mat' in x else x)
noncoding_elements = list(df[df['Final filter judgement'] == True]['ID'].unique())

In [ ]:
file_df = pd.read_csv('data/PCAWG_mutations_to_elements.icgc.public.txt', sep='\t')
file_df

In [21]:
file_df['reg_id'].value_counts()

gc19_pc.cds::gencode::TP53::ENSG00000141510.11           516
gc19_pc.cds::gencode::TTN::ENSG00000155657.19            476
lncrna.ncrna::gencode::SNHG14::ENSG00000224078.8         233
gc19_pc.cds::gencode::KRAS::ENSG00000133703.7            223
lncrna.ncrna::gencode::KCNQ1OT1::ENSG00000269821.1       221
                                                        ... 
gc19_pc.5utr::gencode::GRIK1-AS2::ENSG00000183653.4        1
gc19_pc.5utr::gencode::C5orf46::ENSG00000178776.4          1
gc19_pc.5utr::gencode::ZNF184::ENSG00000096654.11          1
gc19_pc.5utr::gencode::ZNF853::ENSG00000236609.3           1
lncrna.ncrna::gencode::RP11-7O11.3::ENSG00000237950.1      1
Name: reg_id, Length: 108242, dtype: int64

In [ ]:
df[df['Final filter judgement'] == True]['ID'].unique()

In [9]:
rheinbay_noncoding_drivers = file_df[file_df['reg_id'].isin(noncoding_elements)][['mut_chr', 'mut_pos1', 'mut_pos2', 'mut_ref', 'mut_alt', 'region_type', 'reg_id']].drop_duplicates().reset_index(drop=True)
rheinbay_noncoding_drivers.rename({'mut_chr': 'chr', 'mut_pos1': 'start', 'mut_pos2': 'end', 'mut_ref': 'ref', 'mut_alt': 'alt'}, axis=1, inplace=True)
rheinbay_noncoding_drivers['chr'] = rheinbay_noncoding_drivers['chr'].apply(lambda x: x.replace('chr', ''))
rheinbay_noncoding_drivers

,chr,start,end,ref,alt,region_type,reg_id
0,11,111231125,111231125,G,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
1,11,111232885,111232885,C,T,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
2,7,86866372,86866372,T,C,enhancers.bed,enhancers::chr7:86865600-86866400::NA::NA
3,1,2267139,2267139,G,A,enhancers.bed,enhancers::chr1:2267000-2269600::NA::NA
4,11,111226848,111226848,T,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
...,...,...,...,...,...,...,...
901,11,65194294,65194295,-,CATAATATTTTAAAA,lncrna.ncrna.bed,lncrna.ncrna::gencode::NEAT1::ENSG00000245532.4
902,11,65269272,65269273,AA,-,lncrna.ncrna.bed,lncrna.ncrna::gencode::MALAT1::ENSG00000251562.3
903,11,65267659,65267659,A,-,lncrna.ncrna.bed,lncrna.ncrna::gencode::MALAT1::ENSG00000251562.3
904,11,65209144,65209148,TTTAT,-,lncrna.ncrna.bed,lncrna.ncrna::gencode::NEAT1::ENSG00000245532.4


In [29]:
rheinbay_noncoding_drivers[rheinbay_noncoding_drivers['reg_id'].str.contains('TERT')]

,chr,start,end,ref,alt,region_type,reg_id
261,chr5,1295228,1295228,G,A,gc19_pc.promCore.bed,gc19_pc.promCore::gencode::TERT::ENSG000001643...
300,chr5,1295161,1295161,T,G,gc19_pc.promCore.bed,gc19_pc.promCore::gencode::TERT::ENSG000001643...
389,chr5,1295133,1295133,C,T,gc19_pc.promCore.bed,gc19_pc.promCore::gencode::TERT::ENSG000001643...
421,chr5,1295228,1295228,G,T,gc19_pc.promCore.bed,gc19_pc.promCore::gencode::TERT::ENSG000001643...


In [11]:
dr_nod = pd.read_excel('data/Dr. Nod/gkac1251_supplemental_files/SupplementaryTable4.xlsx', engine='openpyxl')
dr_nod = dr_nod[['chr', 'pos0', 'pos1', 'ref', 'alt', 'tissue', 'geneSymbol']]
dr_nod.rename({'pos0': 'start', 'pos1': 'end', 'tissue': 'region_type', 'geneSymbol': 'reg_id'}, inplace=True, axis=1)
dr_nod['chr'] = dr_nod['chr'].apply(lambda x: x.replace('chr', ''))
dr_nod

,chr,start,end,ref,alt,region_type,reg_id
0,1,19578253,19578254,G,A,Brain,MRTO4
1,1,19578254,19578255,G,A,Brain,MRTO4
2,1,23889291,23889292,G,T,Ovary,ID3
3,1,23895459,23895460,G,T,Ovary,ID3
4,1,23888768,23888769,T,C,Ovary,ID3
...,...,...,...,...,...,...,...
210,9,125026995,125026996,C,G,Breast,MRRF
211,9,124049461,124049462,T,A,Breast,MRRF
212,X,70338403,70338404,G,A,Brain,MED12
213,X,70401837,70401838,G,T,Brain,MED12


In [12]:
test_data = pd.concat([rheinbay_noncoding_drivers, dr_nod])
test_data

,chr,start,end,ref,alt,region_type,reg_id
0,11,111231125,111231125,G,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
1,11,111232885,111232885,C,T,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
2,7,86866372,86866372,T,C,enhancers.bed,enhancers::chr7:86865600-86866400::NA::NA
3,1,2267139,2267139,G,A,enhancers.bed,enhancers::chr1:2267000-2269600::NA::NA
4,11,111226848,111226848,T,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
...,...,...,...,...,...,...,...
210,9,125026995,125026996,C,G,Breast,MRRF
211,9,124049461,124049462,T,A,Breast,MRRF
212,X,70338403,70338404,G,A,Brain,MED12
213,X,70401837,70401838,G,T,Brain,MED12


In [18]:
test_data = test_data.drop_duplicates(subset=['chr', 'start', 'end', 'ref', 'alt']).reset_index(drop=True)
test_data

,chr,start,end,ref,alt,region_type,reg_id
0,11,111231125,111231125,G,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
1,11,111232885,111232885,C,T,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
2,7,86866372,86866372,T,C,enhancers.bed,enhancers::chr7:86865600-86866400::NA::NA
3,1,2267139,2267139,G,A,enhancers.bed,enhancers::chr1:2267000-2269600::NA::NA
4,11,111226848,111226848,T,C,enhancers.bed,enhancers::chr11:111222000-111233000::NA::NA
...,...,...,...,...,...,...,...
1057,9,125026995,125026996,C,G,Breast,MRRF
1058,9,124049461,124049462,T,A,Breast,MRRF
1059,X,70338403,70338404,G,A,Brain,MED12
1060,X,70401837,70401838,G,T,Brain,MED12


In [21]:
test_data['driver'] = 1
test_data.to_csv('data/test_data_final.csv', index=False)

In [ ]:
# elements_list = os.listdir('data/gene-coding-and-non-coding-elements/')
# elements_list.remove('allCombined.zip')
# elements_list.remove('mapFiles')

# arr = []
# df['chr'] = ''
# df['start'] = 0
# df['end'] = 0
# for element in elements_list:
#     print(element)
#     tempdf = df[df['file_name'] == element]
#     file_df = pd.read_csv('data/gene-coding-and-non-coding-elements/'+element, sep='\t', header=None)
#     file_df.columns = ['chrom', 'chromStart','chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
#     tempdf.reset_index(drop=True, inplace=True)
#     for index, row in tempdf.iterrows():
#         arr.append(file_df[file_df['name'] == row['ID']])
#         found_row = file_df[file_df['name'] == row['ID']]
#         # print(row['ID'])
#         df.loc[df['ID'] == row['ID'], 'chr'] = found_row['chrom'].values[0].replace('chr', '')
#         df.loc[df['ID'] == row['ID'], 'start'] = found_row['chromStart'].values[0]
#         df.loc[df['ID'] == row['ID'], 'end'] = found_row['chromEnd'].values[0]

In [ ]:
# arr = []
# df['chr'] = ''
# df['start'] = 0
# df['end'] = 0
# df['ref'] = 0
# df['alt'] = 0
# df['region_type'] = 0
# df['found'] = 0
# file_df = pd.read_csv('data/PCAWG_mutations_to_elements.icgc.public.txt', sep='\t')
# for index, row in df.iterrows():
#     found_row = file_df[file_df['reg_id'] == row['ID']]
#     if len(found_row) > 1:
#         print("FOUND", len(found_row))
#     for i, r in found_row.iterrows():
#         print(r[['mut_chr', 'mut_pos1', 'mut_pos2', 'mut_ref', 'mut_alt', 'region_type']])
#     if len(found_row) > 0:
#         df.loc[df['ID'] == row['ID'], 'chr'] = found_row['mut_chr'].values[0].replace('chr', '')
#         df.loc[df['ID'] == row['ID'], 'start'] = found_row['mut_pos1'].values[0]
#         df.loc[df['ID'] == row['ID'], 'end'] = found_row['mut_pos2'].values[0]
#         df.loc[df['ID'] == row['ID'], 'ref'] = found_row['mut_ref'].values[0]
#         df.loc[df['ID'] == row['ID'], 'alt'] = found_row['mut_alt'].values[0]
#         df.loc[df['ID'] == row['ID'], 'region_type'] = found_row['region_type'].values[0]
#         df.loc[df['ID'] == row['ID'], 'found'] = len(found_row)
#     break

In [ ]:
raw_data = pd.read_csv('data/dataset_uncensored.csv')
raw_data['start'] = raw_data['start'].astype(int)
raw_data['end'] = raw_data['end'].astype(int)

In [ ]:
overlaps = check_overlaps(df, raw_data)

In [ ]:
overlaps['target'].value_counts()

In [ ]:
arr_ids = []
arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel'))
arr_ids  = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/snv_mnv'))
# arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/graylist'))
arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/indel'))
arr_ids  = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/snv_mnv'))
# arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/graylist'))
arr_ids = [s for s in arr_ids if '.tbi' not in s]
arr_ids_names = [s[:s.find(".")] for s in arr_ids]

In [ ]:
def read_vcf_gzip(filename):
    with gzip.open(filename, 'r') as f:
        lines = [l for l in f if not l.startswith(b'##')]
    df = pd.read_csv(
        io.StringIO(b''.join(lines).decode()),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
                'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return df
    
read_vcf_gzip('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel/0009b464-b376-4fbc-8a56-da538269a02f.consensus.20161006.somatic.indel.vcf.gz')

In [ ]:
all_files = [val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk('D:\Masters Thesis\GitHub Repo\data\gene-coding-and-non-coding-elements')] for val in sublist]
all_files = [s for s in all_files if 'allCombined' not in s]
all_files = [s for s in all_files if 'mapFiles' not in s]
all_files

In [ ]:
import re
all_df = pd.DataFrame()
all_df['element'] = ''
for file in all_files:
    temp_df = pd.read_csv(file, sep='\t', header=None)
    result = re.search('elements\\\\(.*).bed', file)
    temp_df['element'] = str(result.group(1))
    all_df = pd.concat([all_df, temp_df])
all_df.columns = ['element', 'chrom', 'chromStart','chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
all_df['chromStart'] = all_df['chromStart'].astype(int)
all_df['chromEnd'] = all_df['chromEnd'].astype(int)
all_df['score'] = all_df['score'].astype(int)
all_df['chrom'] = all_df['chrom'].apply(lambda x: x.replace('chr', ''))

all_df.rename({'chrom': 'chr', 'chromStart': 'start', 'chromEnd': 'end'}, inplace=True, axis = 1)

In [ ]:
overlaps = check_overlaps_and_annotate(raw_data, all_df, 'element')

In [ ]:
overlaps[~overlaps['annotation'].isna()]

In [ ]:
overlaps1 = check_overlaps(all_df, raw_data)

In [ ]:
overlaps1['element'].value_counts()

In [ ]:
def row_count(input):
    with open(input) as f:
        for i, l in enumerate(f):
            pass
    return i

row_count('data/Rheinbay 2020/final_consensus_passonly.snv_mnv_indel.icgc.controlled.maf')